# Insights on Hacker News Posts


In this project, we'll work with a data set of submissions to popular technology site Hacker News.

Hacker News is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:


* *id:* The unique identifier from Hacker News for the post

* *title:* The title of the post

* *url:* The URL that the posts links to, if it the post has a URL

* *num_points:* The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
* *num_comments:* The number of comments that were made on the post
* *author:* The username of the person who submitted the post
* *created_at:* The date and time at which the post was submitted

In [1]:
"""
This project uses basic Python techniques instead of other Python libraries
like Numpy, Pandas in order to better grasp programming techniques and algorithms.
"""
from csv import reader 

opened_file = open("datasets/hacker_news.csv") # Open
read_file = reader(opened_file)       # And read the file
hn = list(read_file)                  # List it into HN
headers = hn[0]                       # Take the column row as headers
hn = hn[1:]                           # The data itself into HN

print(hn[0])
print("\n")                           # \n to separate lines for more readability
print(hn[1])
print("\n")
print(hn[2:5])

['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


[['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


We've removed the headers from hn to separate the data itself from headers. For our analysis, we're only interested in `ASK HN` and `SHOW HN` questions. Therefore we're going to extract those two types of questions, and we're going to create a new list containing just the data for those titles.

To find `ASK HN` and `SHOW HN` posts, we'll use the string method `startswith`. 

In [2]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:  # For every row in HN we're gonna look at their labels and append our findings to their separate lists.
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
print("The length of ask posts is {}, \nshow_posts is {}, \nother posts is {}"
      .format(len(ask_posts),len(show_posts),len(other_posts)))

The length of ask posts is 1744, 
show_posts is 1162, 
other posts is 17194


Now that we have separate lists for ask posts and show posts, we can use these to determine whether one type of post receives more comments on average than the other.

In [3]:
total_ask_comments = 0

for row in ask_posts:    # Extract the ammount of comments and append it to it's respective list
    comments = int(row[4]) # Take the number of comments
    total_ask_comments += comments # Add it to the total number of comments for ask hn posts.
    
avg_ask_comments = total_ask_comments / len(ask_posts) # The total number of comments divided by the total number of posts.
print("The average number of comments ask posts get: " + str(avg_ask_comments))

total_show_comments = 0

for row in show_posts: # Same for show_posts.
    comments = int(row[4])
    total_show_comments += comments
    
avg_show_comments = total_show_comments / len(show_posts)
print("The average number of comments show posts get: " + str(avg_show_comments))



The average number of comments ask posts get: 14.038417431192661
The average number of comments show posts get: 10.31669535283993


The average number of comments for ask posts is %40 (4 comments per post) higher.

This gap is a significant difference demonstrating ask posts are more engaging on average than show posts. Since ask posts are more likely to receive comments, we are focusing our efforts to analyze just these posts.

To continue our findings, we're going to delve deeper to find out if ask posts created at a particular time are more likely to attract comments.

To do that, we're going to 
* calculate the number of ask posts created in each hour of the day, along with the number of comments it received.
* calculate the average number of comments ask posts receive by the hour created.

In [4]:
import datetime as dt
result_list = []

for row in ask_posts: # Extracting the time
    created_at = dt.datetime.strptime(row[6], "%m/%d/%Y %H:%M") # Converting to Datetime type
    num_comments = int(row[4]) # Number of comments
    result_list.append([created_at, num_comments]) # Both the amount and time is appended.
    
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:                       # To value count the hours by comments
    hour = dt.datetime.strftime(row[0], "%H") # Extract hour
    if hour not in counts_by_hour:            # If hour doesn't exist in the dictionary
        counts_by_hour[hour] = 1              # Create it
        comments_by_hour[hour] = row[1]       
    else:                                     # If it does, append
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]
        
"""
We created two dictionaries:
counts_by_hour: contains the number of ask posts created
during each hour of the day.
comments_by_hour: contains the corresponding
number of comments ask posts created at each hour received.
Next, we'll use these two dictionaries to calculate
the average number of comments for posts created during each hour of the day.
"""
avg_by_hour = []
for hour1 in counts_by_hour:          # For the hours in counts_by_hour
    for hour2 in comments_by_hour:    # And the hours in comments by hour
        if hour1 == hour2:            # When they're the same 
            avg_by_hour.append([hour1,comments_by_hour[hour2]/counts_by_hour[hour1]]) # Append their average
            
print(sorted(avg_by_hour))

[['00', 8.127272727272727], ['01', 11.383333333333333], ['02', 23.810344827586206], ['03', 7.796296296296297], ['04', 7.170212765957447], ['05', 10.08695652173913], ['06', 9.022727272727273], ['07', 7.852941176470588], ['08', 10.25], ['09', 5.5777777777777775], ['10', 13.440677966101696], ['11', 11.051724137931034], ['12', 9.41095890410959], ['13', 14.741176470588234], ['14', 13.233644859813085], ['15', 38.5948275862069], ['16', 16.796296296296298], ['17', 11.46], ['18', 13.20183486238532], ['19', 10.8], ['20', 21.525], ['21', 16.009174311926607], ['22', 6.746478873239437], ['23', 7.985294117647059]]


That was tedious. But we have what we've wanted! A list with an average number of comments for posts created during each hour of the day. However, even though we sorted the data, it sorts only the index value stores as position 0. What we want is a new list with the average number of comments per post at position 0. Let's swap the positions of our list:

In [5]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]])
# We'd like a descending sort so we're going to set an argument this time
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
print("Top 5 Hours for Ask Posts Comments\n")

for row in sorted_swap[:5]:
    dt_obj = dt.datetime.strptime(row[1], "%H")
    print("{}: {:.2f} average comments per post".format(dt.datetime.strftime(dt_obj, "%H:%M"),row[0]))

Top 5 Hours for Ask Posts Comments

15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post


Voila! The problem is these hours are for Eastern US timezone. I live in Istanbul, which is in GMT +3 timezone. GMT +3 is 7 hours ahead of Eastern US time. So to understand how the results apply to me, I'm going to add 7 hours delay to the results.

In [6]:
for row in sorted_swap[:5]:
    dt_obj = dt.datetime.strptime(row[1], "%H")
    dt_obj = dt.timedelta(hours=7) + dt_obj # With time delta we specify we want to add 7 hours
    print("{}: {:.2f} average comments per post".format(dt.datetime.strftime(dt_obj, "%H:%M"),row[0]))

22:00: 38.59 average comments per post
09:00: 23.81 average comments per post
03:00: 21.52 average comments per post
23:00: 16.80 average comments per post
04:00: 16.01 average comments per post


## Conclusions


So there it is! 

*The top hours (in Central European Time) are:*

* 21:00 - 22:00: 38.59 average comments per post
* 08:00 - 09:00: 23.81 average comments per post
* 02:00 - 03:00: 21.52 average comments per post

These are the hours I should be creating my posts to maximize my engagement off of them. Late in the night before sleep, consecutively at 22, 23. Or first thing in the morning at 9. The late-night hours aren't convenient for me: 03, 04.

Keep in mind, though, the North American people disagree with me on this. As the afternoon hours of 3 and 4 pm are the best time for them to post, followed by the evening at 20, 21. If they're late, 2 am only the second to best option!

The second conclusion is that tagging `ASK HN` over `SHOW HN` is better for attracting comments.

* Ask HN: 14.04 average comments per post
* Show HN: 10.32 average comments per post